# Generate data using api (for now, DR dataset only)

In [60]:
import random
from update_prompt import update_prompt_with_csv


prompt_path = r".\prompts\prompt_complete_DR.txt"
csv_valid_path = r".\data\released_data_mentalllama\train_data\complete_data\DR\reddit_valid.csv"
csv_train_path = r".\data\released_data_mentalllama\train_data\complete_data\DR\reddit_train.csv"

n = 15

In [67]:
df = pd.read_csv(r".\data\released_data_mentalllama\train_data\complete_data\DR\reddit_valid.csv")

df.question.unique()

array([' Question: The answer to the question "Does the poster suffers from depression?" is '],
      dtype=object)

#### data generation

In [8]:
from openai import OpenAI

client_ds = OpenAI(api_key="sk-ed8f7a41078845a5bf6d6db82686ca08", base_url="https://api.deepseek.com")

In [ ]:
import csv
import io
import os

header = ['post', 'question', 'response']
output_csv = './data/data_generated/DR_generated.csv'

for i in range(200):
    try:
        # Update the prompt with new CSV data
        update_prompt_with_csv(prompt_path, csv_valid_path, csv_train_path, n)

        # Read the updated prompt
        with open('./prompts/prompt_complete_DR.txt', 'r') as file:
            content = file.read()

        # Generate data using the updated prompt
        response = client_ds.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": "You are a mental health data generation assistant"},
                {"role": "user", "content": content},
            ],
            stream=False,
            temperature=1.5,
            max_tokens=8192
        )

        # Parse the response and write to CSV
        reader = csv.reader(io.StringIO(response.choices[0].message.content))
        rows = list(reader)

        # Check if the output CSV file exists and write data
        file_exists = os.path.exists(output_csv)
        with open(output_csv, 'a' if file_exists else 'w', newline='', encoding='utf-8') as out_f:
            writer = csv.writer(out_f)
            if not file_exists or os.stat(output_csv).st_size == 0:
                writer.writerow(header)
            writer.writerows(rows)

        print(f"CSV file '{output_csv}' updated with {len(rows)} new rows of data.")

    except Exception as e:
        print(f"Error occurred in iteration {i}: {e}")
        continue

Updated '.\prompts\prompt_complete_DR.txt' with 15 new example rows.


In [69]:
# data prepossessing
df_g = pd.read_csv('./data/data_generated/DR_generated.csv')

In [71]:
df_g

,post,question,response
0,Post: I feel like I'm stuck in a loop that I c...,Question: The answer to the question \Does the...,yes. Reasoning: The post exhibits several emot...
1,"Post: I don't know why, but I feel so empty al...",Question: The answer to the question \Does the...,yes. Reasoning: The post conveys a deep sense ...
2,Post: I keep having these thoughts that I can'...,Question: The answer to the question \Does the...,yes. Reasoning: The post exhibits strong emoti...
3,Post: I feel like I'm watching my life pass me...,Question: The answer to the question \Does the...,yes. Reasoning: The post conveys a deep sense ...
4,"Post: I feel so alone, even when I'm surrounde...",Question: The answer to the question \Does the...,yes. Reasoning: The post exhibits strong emoti...
5,Post: I feel like I'm stuck in a loop. Every d...,Question: The answer to the question Does the ...,yes. Reasoning: The post conveys a strong sens...
6,Post: I can't seem to find joy in anything any...,Question: The answer to the question 'Does the...,yes. Reasoning: The post exhibits several key ...
7,"Post: I’ve been feeling really anxious lately,...",Question: The answer to the question 'Does the...,no. Reasoning: The post primarily describes sy...
8,Post: I’ve been feeling really down for the pa...,Question: The answer to the question 'Does the...,yes. Reasoning: The post clearly describes sym...
9,Post: I’ve been feeling really stressed out la...,Question: The answer to the question 'Does the...,no. Reasoning: The post primarily describes sy...


In [ ]:
df_g.question.unique()

### preprocessing
1. question column
1. delete duplicate
1. delete rows != 3 features
1. delete last row